# Bibliotecas

In [1]:
import os
import tarfile
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, col

# Caminho para os arquivos do projeto

## Área de trabalho do projeto


In [2]:
drive.mount('/content/drive')
workspace_path = '/content/drive/MyDrive/Workspace'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dados brutos

In [3]:
raw_data_path = f'{workspace_path}/fiap/pos-tech/data-analytics/fase-4/dados'

# Sessão Spark

In [4]:
spark = SparkSession.builder.appName('etl').getOrCreate()

# Carga dos dados brutos

## Carga dos dados em um dataframe Spark

In [5]:
# Função para ajustar o nome das colunas do dataframe
def to_snake_case(column_name):
    return column_name.strip().replace('-', '').replace(' ', '_') \
                      .replace('__', '_').lower()

In [6]:
# Pasta temporária
temp_path = 'tmp'
os.makedirs(temp_path, exist_ok=True)

In [7]:
# Extraindo os CSVs dos arquivo .tar.gz
for filename in os.listdir(raw_data_path):
    if filename.endswith(".tar.gz"):
        tar_path = os.path.join(raw_data_path, filename)
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=temp_path)

In [8]:
# Carregando os CSVs no dataframe Spark
df = spark.read \
          .option('header', True) \
          .option('sep', ';') \
          .option('decimal', ',') \
          .option('inferSchema', True) \
          .csv(f'{temp_path}/*.csv')

In [9]:
# Ajustando o nome das colunas
for col_name in df.columns:
    df = df.withColumnRenamed(col_name, to_snake_case(col_name))

In [10]:
print("Número de linhas:", df.count())
df.printSchema()
df.show(1, truncate=False, vertical=True)

Número de linhas: 4401370
root
 |-- regiao_sigla: string (nullable = true)
 |-- estado_sigla: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- revenda: string (nullable = true)
 |-- cnpj_da_revenda: string (nullable = true)
 |-- nome_da_rua: string (nullable = true)
 |-- numero_rua: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- data_da_coleta: string (nullable = true)
 |-- valor_de_venda: string (nullable = true)
 |-- valor_de_compra: string (nullable = true)
 |-- unidade_de_medida: string (nullable = true)
 |-- bandeira: string (nullable = true)

-RECORD 0--------------------------------------
 regiao_sigla      | N                         
 estado_sigla      | AC                        
 municipio         | RIO BRANCO                
 revenda           | AUTO POSTO AMAPA - EIRELI 
 cnpj_da_revenda   |  00.529.581/0001-53     